# 14장 순환 신경망

- 14.1 순환 뉴런
    - 14.1.1 메모리 셀
    - 14.1.2 입력과 출력 시퀀스
- 14.2 텐서플로로 기본 RNN 구성하기
    - 14.2.1 정적으로 타임 스텝 펼치기
    - 14.2.2 동적으로 타임 스텝 펼치기
    - 14.2.3 가변 길이 입력 시퀀스 다루기
    - 14.2.4 가변 길이 출력 시퀀스 다루기
- 14.3 RNN 훈련하기
    - 14.3.1 시퀀스 분류기 훈련하기
    - 14.3.2 시계열 예측을 위해 훈련하기
    - 14.3.3 RNN의 창조성
- 14.4 심층 RNN
    - 14.4.1 여러 GPU에 심층 RNN 분산하기
    - 14.4.2 드롭아웃 적용하기
    - 14.4.3 많은 타임 스텝에서 훈련의 어려움
- 14.5 LSTM 셀
    - 14.5.1 핍홀 연결
- 14.6 GRU 셀
- 14.7 자연어 처리
    - 14.7.1 워드 임베딩
    - 14.7.2 기계 번역을 위한 인코더-디코더 네트워크

기본설정

In [1]:
import numpy as np
import os
import matplotlib
import matplotlib.pyplot as plt

import tensorflow as tf 

def reset_graph(seed=42):
    tf.reset_default_graph()
    tf.set_random_seed(seed)
    np.random.seed(seed)

matplotlib.rc('font', family='NanumBarunGothic')
plt.rcParams['axes.unicode_minus'] = False

In [2]:
## 텐서보드 쥬피터 안에 그리기
from IPython.display import clear_output, Image, display, HTML
import numpy as np    

def strip_consts(graph_def, max_const_size=32):
    """Strip large constant values from graph_def."""
    strip_def = tf.GraphDef()
    for n0 in graph_def.node:
        n = strip_def.node.add() 
        n.MergeFrom(n0)
        if n.op == 'Const':
            tensor = n.attr['value'].tensor
            size = len(tensor.tensor_content)
            if size > max_const_size:
                tensor.tensor_content = "<stripped %d bytes>"%size
    return strip_def

def show_graph(graph_def, max_const_size=32):
    """Visualize TensorFlow graph."""
    if hasattr(graph_def, 'as_graph_def'):
        graph_def = graph_def.as_graph_def()
    strip_def = strip_consts(graph_def, max_const_size=max_const_size)
    code = """
        <script>
          function load() {{
            document.getElementById("{id}").pbtxt = {data};
          }}
        </script>
        <link rel="import" href="https://tensorboard.appspot.com/tf-graph-basic.build.html" onload=load()>
        <div style="height:600px">
          <tf-graph-basic id="{id}"></tf-graph-basic>
        </div>
    """.format(data=repr(str(strip_def)), id='graph'+str(np.random.rand()))

    iframe = """
        <iframe seamless style="width:1200px;height:620px;border:0" srcdoc="{}"></iframe>
    """.format(code.replace('"', '&quot;'))
    display(HTML(iframe))

## 14.1 순환 뉴런

### 14.1.1 메모리 셀

### 14.1.2 입력과 출력 시퀀스

## 14.2 텐서플로로 기본 RNN 구성하기

In [3]:
reset_graph()

In [4]:
n_inputs = 3
n_neurons = 5

X0 = tf.placeholder(tf.float32, [None, n_inputs])
X1 = tf.placeholder(tf.float32, [None, n_inputs])

In [5]:
Wx = tf.Variable(tf.random_normal(shape=[n_inputs, n_neurons], dtype=tf.float32))
Wy = tf.Variable(tf.random_normal(shape=[n_neurons, n_neurons], dtype=tf.float32))
b = tf.Variable(tf.zeros([1, n_neurons], dtype=tf.float32))

In [6]:
Y0 = tf.tanh(tf.matmul(X0, Wx) + b)
Y1 = tf.tanh(tf.matmul(Y0, Wy) + tf.matmul(X1, Wx) + b)

In [7]:
init = tf.global_variables_initializer()

In [8]:
#미니배치:                  샘플0,    샘플1,   샘플2,   샘플3
X0_batch = np.array([[0, 1, 2], [3, 4, 5], [6, 7, 8], [9, 0, 1]]) # t =0
X1_batch = np.array([[9, 8, 7], [0, 0, 0], [6, 5, 4], [3, 2, 1]]) # t =1

In [9]:
with tf.Session() as sess:
    init.run()
    Y0_val, Y1_val = sess.run([Y0, Y1], feed_dict={X0: X0_batch, X1:X1_batch})

In [10]:
print(Y0_val)
print(Y1_val)

[[-0.0664006   0.9625767   0.68105793  0.7091854  -0.898216  ]
 [ 0.9977755  -0.719789   -0.9965761   0.9673924  -0.9998972 ]
 [ 0.99999774 -0.99898803 -0.9999989   0.9967762  -0.9999999 ]
 [ 1.         -1.         -1.         -0.99818915  0.9995087 ]]
[[ 1.         -1.         -1.          0.4020025  -0.9999998 ]
 [-0.12210419  0.62805265  0.9671843  -0.9937122  -0.2583937 ]
 [ 0.9999983  -0.9999994  -0.9999975  -0.85943305 -0.9999881 ]
 [ 0.99928284 -0.99999815 -0.9999058   0.9857963  -0.92205757]]


### 14.2.1 정적으로 타임 스텝 펼치기

In [11]:
n_inputs = 3
n_neurons = 5

In [12]:
reset_graph()

X0 = tf.placeholder(tf.float32, [None, n_inputs])
X1 = tf.placeholder(tf.float32, [None, n_inputs])

In [13]:
basic_cell = tf.contrib.rnn.BasicRNNCell(num_units=n_neurons)
output_seqs, states = tf.contrib.rnn.static_rnn(basic_cell, [X0, X1],
                                                   dtype=tf.float32)

Instructions for updating:
This class is equivalent as tf.keras.layers.SimpleRNNCell, and will be replaced by that in Tensorflow 2.0.


In [14]:
Y0, Y1 = output_seqs

In [15]:
init=tf.global_variables_initializer()

In [22]:
X0_batch = np.array([[0, 1, 2], [3, 4, 5], [6, 7, 8], [9, 0, 1]]) # t =0
X1_batch = np.array([[9, 8, 7], [0, 0, 0], [6, 5, 4], [3, 2, 1]]) # t =1

with tf.Session() as sess:
    init.run()
    Y0_val, Y1_val, S = sess.run([Y0, Y1, states], feed_dict={X0: X0_batch, X1: X1_batch})

In [23]:
print(Y0_val)
print(Y1_val)
print(S)

[[ 0.30741334 -0.32884315 -0.6542847  -0.9385059   0.52089024]
 [ 0.99122757 -0.9542541  -0.7518079  -0.9995208   0.9820235 ]
 [ 0.9999268  -0.99783254 -0.8247353  -0.9999963   0.99947774]
 [ 0.996771   -0.68750614  0.8419969   0.9303911   0.8120684 ]]
[[ 0.99998885 -0.99976057 -0.0667929  -0.9999803   0.99982214]
 [-0.6524943  -0.51520866 -0.37968948 -0.5922594  -0.08968379]
 [ 0.99862397 -0.99715203 -0.03308626 -0.9991566   0.9932902 ]
 [ 0.99681675 -0.9598194   0.39660627 -0.8307606   0.79671973]]
[[ 0.99998885 -0.99976057 -0.0667929  -0.9999803   0.99982214]
 [-0.6524943  -0.51520866 -0.37968948 -0.5922594  -0.08968379]
 [ 0.99862397 -0.99715203 -0.03308626 -0.9991566   0.9932902 ]
 [ 0.99681675 -0.9598194   0.39660627 -0.8307606   0.79671973]]


시퀀스 패딩

In [24]:
n_steps = 2
n_inputs = 3
n_neurons = 5

In [28]:
reset_graph()

X = tf.placeholder(tf.float32, [None, n_steps, n_inputs])
X_seqs = tf.unstack(tf.transpose(X, perm=[1, 0, 2]))

basic_cell = tf.contrib.rnn.BasicRNNCell(num_units=n_neurons)
output_seqs, states = tf.contrib.rnn.static_rnn(basic_cell, X_seqs,
                                                   dtype=tf.float32)
outputs = tf.transpose(tf.stack(output_seqs), perm=[1, 0, 2])

In [29]:
init = tf.global_variables_initializer()

In [31]:
X_batch = np.array([
        # t= 0, t =1
        [[0, 1, 2],[9, 8 ,7]],
        [[3, 4, 5],[0, 0, 0]],
        [[6, 7, 8],[6, 5, 4]],
        [[9, 0, 1],[3, 2, 1]],
    ])

with tf.Session() as sess:
    init.run()
    outputs_val = outputs.eval(feed_dict={X: X_batch})

In [32]:
print(outputs_val)

[[[-0.45652324 -0.68064123  0.40938237  0.63104504 -0.45732826]
  [-0.9428799  -0.9998869   0.94055814  0.9999985  -0.9999997 ]]

 [[-0.8001535  -0.9921827   0.7817797   0.9971032  -0.9964609 ]
  [-0.637116    0.11300927  0.5798437   0.4310559  -0.6371699 ]]

 [[-0.93605185 -0.9998379   0.9308867   0.9999815  -0.99998295]
  [-0.9165386  -0.9945604   0.896054    0.99987197 -0.9999751 ]]

 [[ 0.9927369  -0.9981933  -0.55543643  0.9989031  -0.9953323 ]
  [-0.02746338 -0.73191994  0.7827872   0.9525682  -0.9781773 ]]]


In [34]:
print(np.transpose(outputs_val, axes=[1, 0, 2])[1])

[[-0.9428799  -0.9998869   0.94055814  0.9999985  -0.9999997 ]
 [-0.637116    0.11300927  0.5798437   0.4310559  -0.6371699 ]
 [-0.9165386  -0.9945604   0.896054    0.99987197 -0.9999751 ]
 [-0.02746338 -0.73191994  0.7827872   0.9525682  -0.9781773 ]]


In [37]:
np.transpose(outputs_val, axes=[1, 0, 2]).shape

(2, 4, 5)

### 14.2.2 동적으로 타임 스텝 펼치기

In [38]:
n_steps = 2
n_inputs = 3
n_neurons = 5

In [39]:
reset_graph()

X = tf.placeholder(tf.float32, [None, n_steps, n_inputs])

basic_cell = tf.contrib.rnn.BasicRNNCell(num_units=n_neurons)
outputs, states = tf.nn.dynamic_rnn(basic_cell, X, dtype=tf.float32)

In [40]:
init = tf.global_variables_initializer()

In [41]:
X_batch = np.array([
        [[0, 1, 2], [9, 8, 7]],
        [[3, 4, 5], [0, 0, 0]],
        [[6, 7, 8], [6, 5, 4]],
        [[9, 0, 1], [3, 2, 1]],
])

In [42]:
with tf.Session() as sess:
    init.run()
    outputs_val = outputs.eval(feed_dict={X: X_batch})

In [43]:
print(outputs_val)

[[[-0.85115266  0.87358344  0.5802911   0.8954789  -0.0557505 ]
  [-0.999996    0.99999577  0.9981815   1.          0.37679607]]

 [[-0.9983293   0.9992038   0.98071456  0.999985    0.25192663]
  [-0.7081804  -0.0772338  -0.85227895  0.5845349  -0.78780943]]

 [[-0.9999827   0.99999535  0.9992863   1.          0.5159072 ]
  [-0.9993956   0.9984095   0.83422637  0.99999976 -0.47325212]]

 [[ 0.87888587  0.07356028  0.97216916  0.9998546  -0.7351168 ]
  [-0.9134514   0.3600957   0.7624866   0.99817705  0.80142   ]]]


### 14.2.3 가변 길이 입력 시퀀스 다루기

In [45]:
n_steps = 2
n_inputs = 3
n_neurons = 5

reset_graph()

X = tf.placeholder(tf.float32, [None, n_steps, n_inputs])
basic_cell = tf.contrib.rnn.BasicRNNCell(num_units=n_neurons)

In [46]:
seq_length = tf.placeholder(tf.int32, [None])
outputs, states = tf.nn.dynamic_rnn(basic_cell, X, dtype=tf.float32,
                                       sequence_length=seq_length)

In [48]:
init = tf.global_variables_initializer()

In [49]:
X_batch = np.array([
        [[0, 1, 2],[9, 8, 7]], # 샘플 0
        [[3, 4, 5],[0, 0, 0]], # 샘플 1 (0 벡터로 패딩됨)
        [[6, 7, 8],[6, 5, 4]], # 샘플 2
        [[9, 0, 1],[3, 2, 1]], # 샘플 3
])

seq_length_batch = np.array([2, 1, 2, 2])

In [50]:
with tf.Session() as sess:
    init.run()
    outputs_val, states_val = sess.run(
            [outputs, states], feed_dict={X: X_batch, seq_length: seq_length_batch})

In [53]:
print(outputs_val, end='\n\n\n')
print(states_val)

[[[-0.9123188   0.16516446  0.5548655  -0.39159346  0.20846416]
  [-1.          0.956726    0.99831694  0.99970174  0.96518576]]

 [[-0.9998612   0.6702289   0.9723653   0.6631046   0.74457586]
  [ 0.          0.          0.          0.          0.        ]]

 [[-0.99999976  0.8967997   0.9986295   0.9647514   0.93662   ]
  [-0.9999526   0.9681953   0.96002865  0.98706263  0.85459226]]

 [[-0.96435434  0.99501586 -0.36150697  0.9983378   0.999497  ]
  [-0.9613586   0.9568762   0.7132288   0.97729224 -0.0958299 ]]]


[[-1.          0.956726    0.99831694  0.99970174  0.96518576]
 [-0.9998612   0.6702289   0.9723653   0.6631046   0.74457586]
 [-0.9999526   0.9681953   0.96002865  0.98706263  0.85459226]
 [-0.9613586   0.9568762   0.7132288   0.97729224 -0.0958299 ]]


여기서  
[[-1.          0.956726    0.99831694  0.99970174  0.96518576]  
 [-0.9998612   0.6702289   0.9723653   0.6631046   0.74457586] # t = 0일때  
 [-0.9999526   0.9681953   0.96002865  0.98706263  0.85459226]  
 [-0.9613586   0.9568762   0.7132288   0.97729224 -0.0958299 ]]  

### 14.2.4 가변 길이 출력 시퀀스 다루기

## 14.3 RNN 훈련하기

### 14.3.1 시퀀스 분류기 훈련하기

In [55]:
reset_graph()

n_steps = 28
n_inputs = 28
n_neurons = 150
n_outputs = 10

learning_rate = 0.001

X = tf.placeholder(tf.float32, [None, n_steps, n_inputs])
y = tf.placeholder(tf.int32, [None])

basic_cell = tf.contrib.rnn.BasicRNNCell(num_units=n_neurons)
outputs, states = tf.nn.dynamic_rnn(basic_cell, X, dtype=tf.float32)

logits = tf.layers.dense(states, n_outputs)
xentropy = tf.nn.sparse_softmax_cross_entropy_with_logits(labels=y, 
                                                                         logits=logits)
loss = tf.reduce_mean(xentropy)

optimizer = tf.train.AdamOptimizer(learning_rate=learning_rate)
training_op = optimizer.minimize(loss)

correct = tf.nn.in_top_k(logits, y, 1)
accuracy = tf.reduce_mean(tf.cast(correct, tf.float32))

init = tf.global_variables_initializer()

data

In [56]:
(X_train, y_train), (X_test, y_test) = tf.keras.datasets.mnist.load_data()

In [62]:
X_train = X_train.astype(np.float32)/255.0
X_test = X_test.astype(np.float32)/255.0

y_train = y_train.astype(np.int32)
y_test = y_test.astype(np.int32)

X_valid, X_train = X_train[:5000], X_train[5000:]
y_valid, y_train = y_train[:5000], y_train[5000:]

In [63]:
def shuffle_batch(X, y, batch_size):
    rnd_idx = np.random.permutation(len(X))
    n_batches = len(X) // batch_size
    for batch_idx in np.array_split(rnd_idx, n_batches):
        X_batch, y_batch = X[batch_idx], y[batch_idx]
        yield X_batch, y_batch

In [64]:
n_epochs = 100
batch_size = 150

with tf.Session() as sess:
    init.run()
    for epoch in range(n_epochs):
        for X_batch, y_batch in shuffle_batch(X_train, y_train, batch_size):
            sess.run(training_op, feed_dict={X: X_batch, y:y_batch})
        acc_batch = accuracy.eval(feed_dict={X: X_batch, y:y_batch})
        acc_valid = accuracy.eval(feed_dict={X: X_valid, y:y_valid})
        
        print(epoch, "배치 데이터 정확도:", acc_batch, "검증 세트 정확도:", acc_valid)

0 배치 데이터 정확도: 0.6333333 검증 세트 정확도: 0.6472
1 배치 데이터 정확도: 0.75333333 검증 세트 정확도: 0.766
2 배치 데이터 정확도: 0.78 검증 세트 정확도: 0.8292
3 배치 데이터 정확도: 0.84 검증 세트 정확도: 0.8584
4 배치 데이터 정확도: 0.86 검증 세트 정확도: 0.8408
5 배치 데이터 정확도: 0.87333333 검증 세트 정확도: 0.8746
6 배치 데이터 정확도: 0.9266667 검증 세트 정확도: 0.9026
7 배치 데이터 정확도: 0.8933333 검증 세트 정확도: 0.902
8 배치 데이터 정확도: 0.94 검증 세트 정확도: 0.9074
9 배치 데이터 정확도: 0.8933333 검증 세트 정확도: 0.9166
10 배치 데이터 정확도: 0.9266667 검증 세트 정확도: 0.9166
11 배치 데이터 정확도: 0.9533333 검증 세트 정확도: 0.9254
12 배치 데이터 정확도: 0.9066667 검증 세트 정확도: 0.9096
13 배치 데이터 정확도: 0.94666666 검증 세트 정확도: 0.9208
14 배치 데이터 정확도: 0.9533333 검증 세트 정확도: 0.9326
15 배치 데이터 정확도: 0.9266667 검증 세트 정확도: 0.928
16 배치 데이터 정확도: 0.94666666 검증 세트 정확도: 0.944
17 배치 데이터 정확도: 0.9533333 검증 세트 정확도: 0.939
18 배치 데이터 정확도: 0.94 검증 세트 정확도: 0.9456
19 배치 데이터 정확도: 0.92 검증 세트 정확도: 0.9392
20 배치 데이터 정확도: 0.94 검증 세트 정확도: 0.9318
21 배치 데이터 정확도: 0.96 검증 세트 정확도: 0.9424
22 배치 데이터 정확도: 0.96 검증 세트 정확도: 0.9394
23 배치 데이터 정확도: 0.9533333 검증 세트 정확도: 0.9546
24 배치 데이터 정확도: 0.94 검증 세

### 14.3.2 시계열 예측을 위해 훈련하기

In [75]:
t_min, t_max = 0, 30
resolution = 0.1

def time_series(t):
    return t * np.sin(t) / 3 + 2 * np.sin(t * 5)

def next_batch(batch_size, n_steps):
    t0 = np.random.rand(batch_size, 1) * (t_max - t_min - n_steps * resolution)
    Ts = t0 + np.arange(0., n_steps + 1) * resolution
    ys = time_series(Ts)
    return ys[:, :-1].reshape(-1, n_steps, 1), ys[:, 1:].reshape(-1, n_steps, 1)

In [76]:
n_steps = 20

In [77]:
X_batch, y_batch = next_batch(1, n_steps)

In [94]:
np.random.rand(1, 1)

array([[0.11216173]])

In [98]:
t_max - t_min - n_steps * resolution

28.0

### 14.3.3 RNN의 창조성

## 14.4 심층 RNN

### 14.4.1 여러 GPU에 심층 RNN 분산하기

### 14.4.2 드롭아웃 적용하기

### 14.4.3 많은 타임 스텝에서 훈련의 어려움

## 14.5 LSTM 셀

### 14.5.1 핍홀 연결

## 14.6 GRU 셀

## 14.7 자연어 처리

### 14.7.1 워드 임베딩

### 14.7.2 기계 번역을 위한 인코더-디코더 네트워크